In [1]:
import json
import csv
import requests
import secrets


In [2]:
def generate_new_token(authorisation_code: str, code_verifier: str) -> dict:
    global ID, SECRET

    url = 'https://myanimelist.net/v1/oauth2/token'
    data = {
        'client_id': ID,
        'client_secret': SECRET,
        'code': authorisation_code,
        'code_verifier': code_verifier,
        'grant_type': 'authorization_code'
    }

    response = requests.post(url, data)
    response.raise_for_status()

    token = response.json()
    response.close()
    print('Token get!')

    with open('token.json', 'w') as file:
        json.dump(token, file, indent=4)
    return token


In [3]:
ID = '8d901c092043ca3feb46566a86dd4673'
SECRET = '05ac1742517c8e4d14ebc6f8cbccf4f77f44ea091f3c6fe234087d304a8573f3'


In [4]:
def get_code_verifier() -> str:
    token = secrets.token_urlsafe(100)
    return token[:128]

code_verifier = code_challenge = get_code_verifier()


In [5]:
def print_url(code_challenge: str):
    global ID
    url = f'https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id={ID}&code_challenge={code_challenge}'
    print(f'Plz clicking here: {url}\n')

print_url(code_challenge)


Plz clicking here: https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=8d901c092043ca3feb46566a86dd4673&code_challenge=UFw5o9BMf68Mo1aMCNLCvvrNx-GtE0g1qqMagrZg3PHyzjm3iq-dVs1_Umq5fdWL3qWzBYLVGUz6us66h1_qir6TOciMiXYqhY2o_iFK4G64dJ0QOr9qqz3K8jtJVdAS



Follow this link and get oauth code. Then input the code.

In [7]:
authorisation_code = input('Plz input the Authorisation Code: ').strip()
token = generate_new_token(authorisation_code, code_verifier)


Token get!


Note: fail to correctly receive the callback of authentication. My original redirect URI is localhost/oauth, however, since I don't have an application on that port, this page won't be loaded. However, I found the returned authorization code in the url. I can build a server on localhost next time and listen for the code

In [8]:
def get_anime_list(year, season, url=''):
    if len(url) == 0:
        url = 'https://api.myanimelist.net/v2/anime/season/{}/{}'.format(
            year, season)
    response = requests.get(url, headers={
        'Authorization': f"Bearer {token['access_token']}"
    })
    response.raise_for_status()
    al = response.json()
    response.close()
    return al


In [21]:
def get_anime_info(anime_id):
    url = 'https://api.myanimelist.net/v2/anime/{}?fields=id,title,main_picture,alternative_titles,start_date,end_date,type,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,genres,my_list_status,num_episodes,start_season,broadcast,source,average_episode_duration,rating,pictures,background,related_anime,related_manga,recommendations,studios,statistics'.format(
        anime_id)
    response = requests.get(url, headers={
        'Authorization': f"Bearer {token['access_token']}"
    })

    response.raise_for_status()
    al = response.json()
    response.close()
    return [al['id'], al['title'], al.get('start_date', ''), al.get('end_date', ''), al.get('mean', ''), al.get('rank', ''), al.get('popularity', ''),
            al.get('num_scoring_users', ''), al.get('media_type', ''), "+".join(
                [genre['name'] for genre in al.get('genres', [])]), al.get('num_episodes', 0),
            al['start_season']['year'], al['start_season']['season'], al.get('broadcast', {}).get(
                'day_of_the_week', ''), al.get('broadcast', {}).get('start_time', ''), al.get('source', ''),
            al.get('average_episode_duration', 0), int(al['statistics']['status']['watching']), int(
                al['statistics']['status']['completed']), int(al['statistics']['status']['on_hold']),
            int(al['statistics']['status']['dropped']), int(al['statistics']['status']['plan_to_watch'])]


In [22]:
def store_anime_info(seasons, years):
    # data = [["id", "title", "start_date", "end_date", "mean", "rank", "popularity", "num_scoring_users", "media_type", "genres", "num_episodes",
    #          "year", "season", "day_of_the_week", "start_time", "source", "average_episode_duration", 'watching', "completed", "on_hold", "dropped", "plan_to_watch"]]
    # with open("anime.csv", "w", encoding='utf-8', newline='') as f:
    #   writer = csv.writer(f)
    #   writer.writerows(data)
    for year in years:
        for season in seasons:
            data = []
            nxt = ''
            Start = False
            while nxt != '' or not Start:
                Start = True
                animes_list = get_anime_list(year, season, nxt)
                for anime in animes_list['data']:
                    data.append(get_anime_info(anime['node']['id']))
                nxt = animes_list.get('paging', {}).get('nxt', '')
            with open("anime.csv", "a", encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(data)


In [23]:
seasons = ['spring', 'summer', 'fall', 'winter']
years = range(2021, 2022)
animes = store_anime_info(seasons, years)


The collected data anime from Myanimelist.

In [4]:
class TreeNode:
    def __init__(self,id,val):
        self.id = id
        self.val = val
        self.left = None
        self.right = None


In [16]:
class Tree:
    def insert(self,root,id,val):
        if root == None:
            root = TreeNode(id,val)
        elif val <= root.val:
            root.left = self.insert(root.left, id, val)
        elif val > root.val:
            root.right = self.insert(root.right, id, val)
        return root
    
    def query(self, root, val):
        if root == None:
            return False
        if root.val == val:
            return True
        elif val < root.val:
            return self.query(root.left, val)
        elif val > root.val:
            return self.query(root.right, val)

    def findMin(self, root):
        if root.left:
            return self.findMin(root.left)
        else:
            return root

    def printTree(self, root):
        if root == None:
            return 
        self.printTree(root.left)
        print(root.id, end = ' ')
        print(root.val, end = '/n')
        self.printTree(root.right) 


In [23]:
import csv
import json
import pandas as pd
csvfile = open('anime.csv', 'r', encoding = "ISO-8859-1")
jsonfile = open('json.json','w')
namesss = pd.read_csv('anime.csv', encoding = "ISO-8859-1")
fieldnames1 = namesss.columns
aaaa = tuple(fieldnames1)
reader = csv.DictReader(csvfile,aaaa)
jsonfile.write('[')
for row in reader:
    json.dump(row,jsonfile,indent=4)
    jsonfile.write(',\n')
jsonfile.write(']')

1